In [1]:
import os
import json

# Folder containing the JSON files
folder_path = "./"  # Update to your folder path
output_file = "combined_dataset.json"

# Initialize an empty list to store data from all JSON files
combined_data = []

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):  # Process only JSON files
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, "r", encoding="utf-8") as json_file:
            data = json.load(json_file)  # Load the content of the JSON file
            # Check if the data is a list or a single object
            if isinstance(data, list):
                combined_data.extend(data)  # Append list data
            else:
                combined_data.append(data)  # Append single object

# Reorder index (assign new unique IDs sequentially)
for idx, item in enumerate(combined_data):
    item['id'] = idx  # Reassign ID to be sequential starting from 0

# Save the combined data to a single JSON file
with open(output_file, "w", encoding="utf-8") as out_file:
    json.dump(combined_data, out_file, indent=4, ensure_ascii=False)

print(f"All JSON files have been combined and reindexed into {output_file}")


All JSON files have been combined into combined_dataset.json


In [2]:
import pandas as pd
import json

# 加载 JSON 文件
input_file = "./original/combined_dataset.json"  # 替换为你的 JSON 文件路径
with open(input_file, "r", encoding="utf-8") as file:
    combined_data = json.load(file)

# 将数据加载到 Pandas DataFrame
df = pd.DataFrame(combined_data)

# 定义每个 subcategory 抽样的数量
sample_size = 100

# 按 'subcategory' 分组，然后从每组中随机抽取指定数量的样本
sampled_data = df.groupby('subcategory', group_keys=False).apply(lambda x: x.sample(min(len(x), sample_size)))

# 重置索引
sampled_data = sampled_data.reset_index(drop=True)

# 将抽取的数据保存到一个新的 JSON 文件
output_file = "Fraud24kRaw.json"
sampled_data.to_json(output_file, orient="records", force_ascii=False, indent=4)

print(f"Sampled data has been saved to {output_file}")


Sampled data has been saved to Fraud24kRaw.json


/tmp/ipykernel_1987/2291331085.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_data = df.groupby('subcategory', group_keys=False).apply(lambda x: x.sample(min(len(x), sample_size)))


In [2]:
import pandas as pd
import json

# 加载抽样后的 JSON 文件
output_file = "Fraud24kRaw.json"  # 替换为保存的抽样 JSON 文件路径
with open(output_file, "r", encoding="utf-8") as file:
    sampled_data = json.load(file)

# 将数据加载到 Pandas DataFrame
df_sampled = pd.DataFrame(sampled_data)

# 总数据量
total_count = len(df_sampled)
print(f"总数据量: {total_count}")

# 按 'subcategory' 统计数量
subcategory_counts = df_sampled['subcategory'].value_counts()
print("\n每个 'subcategory' 的数量:")
print(subcategory_counts)

# 按 'category' 统计数量
category_counts = df_sampled['category'].value_counts()
print("\n每个 'category' 的数量:")
print(category_counts)

category_counts = df_sampled['language'].value_counts()
print("\n每个 'language' 的数量:")
print(category_counts)

# 按 'data_type' 统计数量
if 'data_type' in df_sampled.columns:
    data_type_counts = df_sampled['data_type'].value_counts()
    print("\n每个 'data_type' 的数量:")
    print(data_type_counts)


总数据量: 1236

每个 'subcategory' 的数量:
subcategory
acquaintances                                                       100
e-commerce logistics and shopping                                   100
fake job posting                                                    100
marriage and dating                                                 100
investment and financial management                                 100
ssn                                                                 100
reward                                                              100
public security, prosecution, judiciary, and government agencies    100
refund                                                              100
support                                                             100
commercial spam                                                      80
fraud                                                                80
phishing                                                             76
Name: count, dtype